In [90]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import random
import networkx as nx

Data Loading

In [91]:
ratings= pd.read_csv("./Data/rating.csv", low_memory=False)
train_data = pd.read_csv("./Data/train_data.csv",low_memory=False)
test_data = pd.read_csv("./Data/test_data.csv",low_memory=False)
# train_data, test_data = train_test_split(ratings, test_size=0.3, random_state=42)
# # 保存训练集为csv文件
# train_data.to_csv('./Data/train_data.csv',index=False)
# # 保存测试集为csv文件
# test_data.to_csv('./Data/test_data.csv',index=False)

In [92]:
datasets = ratings.dataset_id.unique()
models = ratings.model_id.unique()
datasets_train = train_data.dataset_id.unique()
model_train = train_data.model_id.unique()
datasets_test = test_data.dataset_id.unique()
model_test = test_data.model_id.unique()

Rating Matrix

In [93]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=model_train)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [94]:
for row in train_data.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]

In [95]:
for row in test_data.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]

Dataset Similarity Matrix

In [96]:
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    u1_ratings = ratings.loc[user1].dropna()
    u2_ratings = ratings.loc[user2].dropna()

    common_items = np.intersect1d(u1_ratings.index, u2_ratings.index).tolist()
    u1_common_ratings = u1_ratings.loc[common_items]
    u2_common_ratings = u2_ratings.loc[common_items]

    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_common_ratings, u2_common_ratings) / (np.linalg.norm(u1_common_ratings) * np.linalg.norm(u2_common_ratings))
        return cos_sim

In [97]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [98]:
def create_bipartite_adjacency_matrix(rating_matrix):
    n_users, n_items = rating_matrix.shape
    adjacency_matrix = np.zeros((n_users + n_items, n_users + n_items))
    adjacency_matrix[:n_users, n_users:] = rating_matrix
    adjacency_matrix[n_users:, :n_users] = rating_matrix.T
    return adjacency_matrix

In [99]:
def propagation_matrix(adjacency, lambda_):
    n = adjacency.shape[0]
    I = np.eye(n)
    # 将 NaN 视为 0
    adjacency = np.nan_to_num(adjacency)
    try:
        P = np.linalg.inv(I - lambda_ * adjacency)
    except np.linalg.LinAlgError:
        print("矩阵不可逆，无法计算传播矩阵")
        return None
    return P

In [100]:
def propagation_matrix_withWalkLength(adjacency_matrix, max_walk_length):
    adjacency_matrix = np.nan_to_num(adjacency_matrix)
    propagation_matrix = np.eye(adjacency_matrix.shape[0])
    sum_matrix = np.eye(adjacency_matrix.shape[0])

    for _ in range(max_walk_length):
        propagation_matrix = propagation_matrix @ adjacency_matrix
        sum_matrix += propagation_matrix

    return sum_matrix

In [101]:
start_time_train = time.time()

In [102]:
# 计算二分图邻接矩阵
bipartite_adjacency_matrix = create_bipartite_adjacency_matrix(data_model_train_matrix)

In [103]:
propagation_maxLength = propagation_matrix_withWalkLength(bipartite_adjacency_matrix, 1)

In [104]:
# 提取用户-商品传播矩阵和商品-用户传播矩阵
n_users = data_model_train_matrix.shape[0]
user_item_propagation = propagation_maxLength[:n_users, n_users:]
item_user_propagation = propagation_maxLength[n_users:, :n_users]

In [105]:
#计算 Random Walk Kernel
random_walk_kernel = np.dot(user_item_propagation, item_user_propagation)

In [106]:
def normalize_kernel(kernel_matrix):
    # 计算矩阵的最小值和最大值
    min_val = np.min(kernel_matrix)
    max_val = np.max(kernel_matrix)

    # 防止除数为零的情况
    if max_val == min_val:
        return np.zeros_like(kernel_matrix)

    # 将矩阵的值缩放到0和1之间
    normalized_kernel_matrix = (kernel_matrix - min_val) / (max_val - min_val)

    return normalized_kernel_matrix

In [107]:
normalized_kernel = normalize_kernel(random_walk_kernel)

In [108]:
normalized_kernel = pd.DataFrame(normalized_kernel,index=datasets_train,columns=datasets_train)

In [109]:
for i in datasets_train:
    for j in datasets_train:
        dataset_similarity.loc[i][j] = normalized_kernel.loc[i][j]

In [110]:
dataset_similarity

,39,7,21,48,12,52,22,51,64,1,...,8,40,63,44,57,53,58,60,59,17
39,0.074704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.018566,0.0,0.032153,0.0,0.0,0.0,0.0,0.0
7,0.0,0.824154,0.0,0.0,0.6334,0.0,0.0,0.613754,0.0,0.504601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.242059,0.0,0.0,0.167552,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18854
48,0.0,0.0,0.0,0.580551,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.6334,0.0,0.0,1.0,0.0,0.0,0.726854,0.0,0.646514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.103514,0.0,...,0.0,0.057457,0.0,0.0,0.0,0.189287,0.0,0.0,0.120647,0.0
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.21565,0.0,0.0,0.288779,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.344424,0.0,0.0
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.123011,0.0,...,0.0,0.06096,0.0,0.0,0.0,0.120647,0.0,0.0,0.199787,0.0


In [111]:
# 计算非零元素的数量
non_zero_count = np.count_nonzero(dataset_similarity)

In [112]:
# 计算所有元素的数量
total_count = dataset_similarity.size

In [113]:
# 计算非零元素的占比
non_zero_ratio = non_zero_count / total_count

In [114]:
non_zero_ratio

0.07175925925925926

In [115]:
end_time_train = time.time()

In [116]:
end_time_train - start_time_train

0.6982159614562988

Prediction

In [117]:
start_time_ref = time.time()

In [118]:
# 预测函数
def predict(ratings, similarity):
    mean_user_rating = ratings.fillna(0).mean(axis=1)
    ratings_diff = (ratings - mean_user_rating[:, np.newaxis]).fillna(0)
    pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
     # 只替换NaN值
    df_nan = ratings.isnull()
    pred = pd.DataFrame(pred).where(df_nan, ratings)
    return pred

In [119]:
user_prediction = predict(data_model_train_matrix,dataset_similarity).sort_index(axis=0).sort_index(axis=1)

c:\users\byy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  after removing the cwd from sys.path.
c:\users\byy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """


In [120]:
end_time_ref = time.time()

In [121]:
end_time_ref - start_time_ref

0.983036994934082

In [122]:
mask = (data_model_test_matrix.fillna(0) != 0) & (user_prediction.fillna(0) != 0)

# 只选择那些在预测评分和实际评分中都不是 0 的评分
prediction = user_prediction[mask].values.flatten()
prediction = pd.to_numeric(prediction, errors='coerce')
prediction = prediction[~np.isnan(prediction)]

In [123]:
actual = data_model_test_matrix.fillna(0)[mask].values.flatten()
actual = pd.to_numeric(actual, errors='coerce')
actual = actual[~np.isnan(actual)]

In [124]:
def calculate_rmse(prediction, actual):
    # 计算 RMSE
    return sqrt(mean_squared_error(prediction, actual))


In [125]:
user_rmse = calculate_rmse(prediction, actual)

In [126]:
user_rmse

0.3234044372118053

In [127]:
def ndcg(y_true, y_pred, k):
    """计算 NDCG @k
    y_true: 真实的 relevancy 分数（通常为 0 或 1）
    y_pred: 预测的 relevancy 分数
    k: 截断位置
    """
    # 计算 DCG @k
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    dcg = np.sum(gains / discounts)

    # 计算 IDCG @k
    ideal_order = np.argsort(y_true)[::-1]
    ideal_gains = 2 ** np.take(y_true, ideal_order[:k]) - 1
    ideal_discounts = np.log2(np.arange(len(ideal_gains)) + 2)
    idcg = np.sum(ideal_gains / ideal_discounts)

    # 防止0除问题
    if idcg == 0:
        return 0

    # 计算 NDCG @k
    ndcg = dcg / idcg
    return ndcg



In [128]:
ndcg(actual, prediction, 10)

0.8945293585430416